## Migration

The v0.4 API is layered: the [Core API](https://microsoft.github.io/autogen/stable/user-guide/core-user-guide/index.html) is the foundation layer offering a scalable, **event-driven actor framework** for creating agentic workflows; the [AgentChat API](https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/index.html) is built on Core, offering a **task-driven, high-level framework** for building interactive agentic applications. It is a replacement for AutoGen v0.2.

## 1.ModelClient

## Use component config

AutoGen 0.4 has a [generic component configuration system](https://microsoft.github.io/autogen/stable/user-guide/core-user-guide/framework/component-config.html). Model clients are a great use case for this. See below for how to create an OpenAI chat completion client.

### Loading a component from a config

In [4]:
import os
from dotenv import load_dotenv
load_dotenv()

from autogen_core.models import ChatCompletionClient

config = {
    "provider": "openai_chat_completion_client",
    "config": {"model": "gpt-4o",
              "api_key": os.getenv("OPENAI_API_KEY")},
}

client = ChatCompletionClient.load_component(config)

ChatCompletionClient.dump_component(client)

ComponentModel(provider='autogen_ext.models.openai.OpenAIChatCompletionClient', component_type='model', version=1, component_version=1, description='Chat completion client for OpenAI hosted models.', label='OpenAIChatCompletionClient', config={'model': 'gpt-4o', 'api_key': SecretStr('**********')})

### Creating a component class

To add component functionality to a given class:

1. Add a call to `Component()` in the class inheritance list.

2. Implment the `_to_config()` and _from_config() methods

For example:

In [9]:
from pydantic import BaseModel
from autogen_core import Component, ComponentBase

# ComponentBase: allows type checking for components without requiring the full Component implementation
# Component: provides the complete functionality for components that need configuration management.

class Config(BaseModel):
    value:str

# MyComponent is inheriting from two base classes
# Both base classes are generic classes parameterized with the Config type
class MyComponent(ComponentBase[Config], Component[Config]):
    component_type = "custom"
    component_config_schema= Config

    def __init__(self, value: str):
        self.value = value

    def _to_config(self)->Config:
        return Config(value=self.value)

    # This is a factory method that creates 
    # a new MyComponent instance from a configuration object
    @classmethod
    def _from_config(cls, config: Config)->"MyComponent":
        return cls(value=config.value)

### Secrets

If a field of a config object is a secret value, it should be marked using [SecretStr](https://docs.pydantic.dev/latest/api/types/#pydantic.types.SecretStr), this will ensure that the value will not be dumped to the config object.

```python
from pydantic import BaseModel, SecretStr


class ClientConfig(BaseModel):
    endpoint: str
    api_key: SecretStr
```

## Use model client class directly

Azure OpenAI:

```python
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient

model_client = AzureOpenAIChatCompletionClient(
    azure_deployment="gpt-4o",
    azure_endpoint="https://<your-endpoint>.openai.azure.com/",
    model="gpt-4o",
    api_version="2024-09-01-preview",
    api_key="sk-xxx",
)
```

OpenAI:

In [10]:
from autogen_ext.models.openai import OpenAIChatCompletionClient

model_client = OpenAIChatCompletionClient(
    model="gpt-4.1-nano",
    api_key=os.getenv("OPENAI_API_KEY")
)

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


### Model Client for OpenAI-Compatible APIs

You can use a the OpenAIChatCompletionClient to connect to an OpenAI-Compatible API, but you need to specify the `base_url` and `model_info`.

```python
from autogen_ext.models.openai import OpenAIChatCompletionClient

custom_model_client = OpenAIChatCompletionClient(
    model="custom-model-name",
    base_url="https://custom-model.com/reset/of/the/path",
    api_key="placeholder",
    model_info={
        "vision": True,
        "function_calling": True,
        "json_output": True,
        "family": "unknown",
        "structured_output": True,
    },
)
```

Read about [Model Clients](https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/tutorial/models.html) in AgentChat Tutorial and more detailed information on [Core API Docs](https://microsoft.github.io/autogen/stable/user-guide/core-user-guide/components/model-clients.html)

Support for other hosted models will be added in the future.

## 2.Model Client Cache

In `v0.4`, the cache is not enabled by default, to use it you need to use a [ChatCompletionCache](https://microsoft.github.io/autogen/stable/reference/python/autogen_ext.models.cache.html#autogen_ext.models.cache.ChatCompletionCache) wrapper around the model client. You can use a [DiskCacheStore](https://microsoft.github.io/autogen/stable/reference/python/autogen_ext.cache_store.diskcache.html#autogen_ext.cache_store.diskcache.DiskCacheStore) or [RedisStore](https://microsoft.github.io/autogen/stable/reference/python/autogen_ext.cache_store.redis.html#autogen_ext.cache_store.redis.RedisStore) to store the cache. `pip install -U "autogen-ext[openai, diskcache, redis]"`

Here’s an example of using `diskcache` for local caching:

In [15]:
import tempfile
from autogen_core.models import UserMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.models.cache import ChatCompletionCache, CHAT_CACHE_VALUE_TYPE
from autogen_ext.cache_store.diskcache import DiskCacheStore
from diskcache import Cache

async def main():
    with tempfile.TemporaryDirectory() as tmpdirname:
        # Initialize the original client
        openai_model_client = OpenAIChatCompletionClient(
            model="gpt-4.1-nano",
            api_key=os.getenv("OPENAI_API_KEY")
        )

        # Then initialize the CacheStore, in this case with diskcache.Cache.
        cache_store = DiskCacheStore[CHAT_CACHE_VALUE_TYPE](Cache(tmpdirname))
        # Create a cache_client wrap around model_client and cache_store
        cache_client = ChatCompletionCache(openai_model_client, cache_store)

        # You can also use redis like:
        # from autogen_ext.cache_store.redis import RedisStore
        # import redis
        # redis_instance = redis.Redis()
        # cache_store = RedisCacheStore[CHAT_CACHE_VALUE_TYPE](redis_instance)

        response = await cache_client.create([
            UserMessage(content="Hello, how are you?", source="user")
        ])
        print(response)  # Should print response from OpenAI


        response = await cache_client.create([
            UserMessage(content="Hello, how are you?", source="user")
        ])
        print(response)  # Should print response from OpenAI

        await openai_model_client.close()

await main()

finish_reason='stop' content="Hello! I'm doing well, thank you. How can I assist you today?" usage=RequestUsage(prompt_tokens=14, completion_tokens=16) cached=False logprobs=None thought=None
finish_reason='stop' content="Hello! I'm doing well, thank you. How can I assist you today?" usage=RequestUsage(prompt_tokens=14, completion_tokens=16) cached=True logprobs=None thought=None


## 3.Assistant Agent

In v0.4, it is similar, but you need to specify `model_client` instead of `llm_config`.

```python
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient

model_client = OpenAIChatCompletionClient(model="gpt-4o", api_key="sk-xxx", seed=42, temperature=0)

assistant = AssistantAgent(
    name="assistant",
    system_message="You are a helpful assistant.",
    model_client=model_client,
)
```

Instead of calling `assistant.send`, you call `assistant.on_messages` or `assistant.on_messages_stream` to handle incoming messages. Furthermore, the on_messages and on_messages_stream methods are asynchronous, and the latter returns an async generator to stream the inner thoughts of the agent.

Here is how you can call the assistant agent in v0.4 directly, continuing from the above example:

In [17]:
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.agents import AssistantAgent
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient

async def main() -> None:

    model_client = OpenAIChatCompletionClient(
        model="gpt-4.1-nano",
        seed=42,
        temperature=0,
        api_key=os.getenv("OPENAI_API_KEY")
    )

    assistant = AssistantAgent(
        name="assistant",
        system_message="You are a helpful assistant",
        model_client=model_client
    )

    cancellation_token = CancellationToken()
    response= await assistant.on_messages(
        [TextMessage(content="Hello!", source="user")],
        cancellation_token
    )
    print(response)

await main()

Response(chat_message=TextMessage(id='cbf6d0fe-b7e5-4f06-b273-ee56a9ec99e2', source='assistant', models_usage=RequestUsage(prompt_tokens=19, completion_tokens=9), metadata={}, created_at=datetime.datetime(2025, 7, 19, 6, 11, 17, 484229, tzinfo=datetime.timezone.utc), content='Hello! How can I assist you today?', type='TextMessage'), inner_messages=[])


The [CancellationToken](https://microsoft.github.io/autogen/stable/reference/python/autogen_core.html#autogen_core.CancellationToken) can be used to cancel the request asynchronously when you call `cancellation_token.cancel()`, which will cause the `await` on the `on_messages` call to raise a CancelledError.

Read more on [Agent Tutorial](https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/tutorial/agents.html) and [AssistantAgent](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.agents.html#autogen_agentchat.agents.AssistantAgent).

Explain the three key methods in the `AssistantAgent` class:

1. `run` (inherited from BaseChatAgent)
**Purpose**: Executes a task and returns the complete result
**Usage**: `result = await agent.run(task="Your task here")`
**Behavior**:
- Runs the agent to completion
- Returns a `TaskResult` containing all messages
- Last message is the final response
- Blocks until the entire response is ready

3. `on_messages`
**Purpose**: Processes incoming messages and generates a response
**Usage**: `response = await agent.on_messages(messages, cancellation_token)`
**Behavior**:
- Processes a sequence of incoming messages
- Returns a single Response object
- Used internally by `run()`
- Handles tool calls and memory updates
- Supports cancellation

5. `on_messages_stream`
**Purpose**: Processes messages and streams the response
**Usage**:
```python
async for event in agent.on_messages_stream(messages, cancellation_token):
    # Process streaming events
    if isinstance(event, Response):
        # Final response
        break
```
**Behavior**:
- Yields events, messages, and the final response
- Supports real-time streaming of model outputs
- Useful for showing progress to users
- Yields ModelClientStreamingChunkEvent for token-by-token streaming
- Final item is always a Response object

## 4.Multi-Modal Agent

e [AssistantAgent](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.agents.html#autogen_agentchat.agents.AssistantAgent) in v0.4 supports multi-modal inputs if the model client supports it. The vision capability of the model client is used to determine if the agent supports multi-modal inputs.

```python
import asyncio
from pathlib import Path
from autogen_agentchat.messages import MultiModalMessage
from autogen_agentchat.agents import AssistantAgent
from autogen_core import CancellationToken, Image
from autogen_ext.models.openai import OpenAIChatCompletionClient

async def main() -> None:
    model_client = OpenAIChatCompletionClient(model="gpt-4o", seed=42, temperature=0)

    assistant = AssistantAgent(
        name="assistant",
        system_message="You are a helpful assistant.",
        model_client=model_client,
    )

    cancellation_token = CancellationToken()
    message = MultiModalMessage(
        content=["Here is an image:", Image.from_file(Path("test.png"))],
        source="user",
    )
    response = await assistant.on_messages([message], cancellation_token)
    print(response)

    await model_client.close()

asyncio.run(main())
```

## 5.User Proxy

In v0.4, a user proxy is simply an agent that takes user input only, there is no other special configuration needed. You can create a user proxy as follows:

See [UserProxyAgent](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.agents.html#autogen_agentchat.agents.UserProxyAgent) for more details and how to customize the input function with timeout.

In [1]:
from autogen_agentchat.agents import UserProxyAgent

user_proxy = UserProxyAgent("user_proxy")

## 6.RAG Agent

In v0.4, you can implement a RAG agent using the [Memory](https://microsoft.github.io/autogen/stable/reference/python/autogen_core.memory.html#autogen_core.memory.Memory) class. Specifically, you can define a memory store class, and pass that as a parameter to the assistant agent. See the [Memory](https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/memory.html) tutorial for more details.

This clear separation of concerns allows you to implement a memory store that uses any database or storage system you want (you have to inherit from the Memory class) and use it with an assistant agent. The example below shows how to use a ChromaDB vector memory store with the assistant agent. In addition, your application logic should determine how and when to add content to the memory store. For example, you may choose to call memory.add for every response from the assistant agent or use a separate LLM call to determine if the content should be added to the memory store.

```python

# ...
# example of a ChromaDBVectorMemory class
chroma_user_memory = ChromaDBVectorMemory(
    config=PersistentChromaDBVectorMemoryConfig(
        collection_name="preferences",
        persistence_path=os.path.join(str(Path.home()), ".chromadb_autogen"),
        k=2,  # Return top  k results
        score_threshold=0.4,  # Minimum similarity score
    )
)

# you can add logic such as a document indexer that adds content to the memory store

assistant_agent = AssistantAgent(
    name="assistant_agent",
    model_client=OpenAIChatCompletionClient(
        model="gpt-4o",
    ),
    tools=[get_weather],
    memory=[chroma_user_memory],
)
```

## 7.Conversable Agent and Register Reply

In `v0.4`, we can simply create a custom agent and implement the `on_messages`, `on_reset`, and `produced_message_types` methods.

```python
from typing import Sequence
from autogen_core import CancellationToken
from autogen_agentchat.agents import BaseChatAgent
from autogen_agentchat.messages import TextMessage, BaseChatMessage
from autogen_agentchat.base import Response

class CustomAgent(BaseChatAgent):
    async def on_messages(self, messages: Sequence[BaseChatMessage], cancellation_token: CancellationToken) -> Response:
        return Response(chat_message=TextMessage(content="Custom reply", source=self.name))

    async def on_reset(self, cancellation_token: CancellationToken) -> None:
        pass

    @property
    def produced_message_types(self) -> Sequence[type[BaseChatMessage]]:
        return (TextMessage,)
```

## 8.Save and Load Agent State

In `v0.4`, you can call `save_state` and `load_state` methods on agents to save and load their state.

You can also call `save_state` and `load_state` on any teams, such as [RoundRobinGroupChat](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.teams.html#autogen_agentchat.teams.RoundRobinGroupChat) to save and load the state of the entire team.

In [4]:
import os
import json
from dotenv import load_dotenv
load_dotenv()

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.agents import AssistantAgent
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient

async def main() -> None:
    
    # Create model_client
    model_client = OpenAIChatCompletionClient(model="gpt-4o",
                                              seed=42,
                                              temperature=0,
                                              api_key=os.getenv("OPENAI_API_KEY"))
    
    # Create assistant
    assistant = AssistantAgent(
        name="assistant",
        system_message="You are a helpful assistant.",
        model_client=model_client,
    )

    cancellation_token = CancellationToken()
    response = await assistant.on_messages(
        [TextMessage(content="Hello!", source="user")], cancellation_token
        )
    print(response)

    # Save the state.
    state = await assistant.save_state()

    # (Optional) Write state to disk.
    with open("assistant_state.json", "w") as f:
        json.dump(state, f)

    # (Optional) Load it back from disk.
    with open("assistant_state.json", "r") as f:
        state = json.load(f)
        print(state) # Inspect the state, which contains the chat history.

    # Carry on the chat.
    response = await assistant.on_messages(
        [TextMessage(content="Tell me a joke.", source="user")], cancellation_token
        )
    print(response)

    # Load the state, resulting the agent to revert to the previous state before the last message.
    await assistant.load_state(state)

    # Carry on the same chat again.
    response = await assistant.on_messages(
        [TextMessage(content="Tell me a joke.", source="user")], cancellation_token
        )
    
    # Close the connection to the model client.
    await model_client.close()

await main()

Response(chat_message=TextMessage(id='4ab14f48-662a-4cc8-a607-fb6da2cc7c7a', source='assistant', models_usage=RequestUsage(prompt_tokens=20, completion_tokens=9), metadata={}, created_at=datetime.datetime(2025, 7, 19, 8, 20, 23, 617337, tzinfo=datetime.timezone.utc), content='Hello! How can I assist you today?', type='TextMessage'), inner_messages=[])
{'type': 'AssistantAgentState', 'version': '1.0.0', 'llm_context': {'messages': [{'content': 'Hello!', 'source': 'user', 'type': 'UserMessage'}, {'content': 'Hello! How can I assist you today?', 'thought': None, 'source': 'assistant', 'type': 'AssistantMessage'}]}}
Response(chat_message=TextMessage(id='22fb56d7-75d1-40bd-8d77-5d4471493e8c', source='assistant', models_usage=RequestUsage(prompt_tokens=43, completion_tokens=14), metadata={}, created_at=datetime.datetime(2025, 7, 19, 8, 20, 24, 542312, tzinfo=datetime.timezone.utc), content="Sure! Why don't scientists trust atoms?\n\nBecause they make up everything!", type='TextMessage'), inn

## 9.Two-Agent Chat

To get the same behavior in `v0.4`, you can use the [AssistantAgent](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.agents.html#autogen_agentchat.agents.AssistantAgent) and [CodeExecutorAgent](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.agents.html#autogen_agentchat.agents.CodeExecutorAgent) together in a [RoundRobinGroupChat](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.teams.html#autogen_agentchat.teams.RoundRobinGroupChat).

In [10]:
from autogen_agentchat.ui import Console
from autogen_agentchat.agents import AssistantAgent, CodeExecutorAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination
from autogen_ext.code_executors.local import LocalCommandLineCodeExecutor
from autogen_ext.models.openai import OpenAIChatCompletionClient

async def main()->None:

    model_client = OpenAIChatCompletionClient(
        model="gpt-4.1-nano",
        seed=42,
        temperature=0,
        api_key=os.getenv("OPENAI_API_KEY")
    )

    assistant = AssistantAgent(
        name="assistant",
        system_message="""
        You are a helpful assistant, Write all code in python.
        Please provide at least one markdown-encoded code block to execute (i.e., quoting code in ```python or ```sh code blocks).
        Reply only 'TERMINATE' if the task is done.
        """,
        model_client=model_client
    )

    code_executor = CodeExecutorAgent(
        name="code_executor",
        code_executor=LocalCommandLineCodeExecutor(work_dir="coding") # executes code locally in your system’s command line (shell).
    )

    # The termination condition is a combination of text termination
    # and max message termination, either of which will cause the chat to terminate.

    termination = TextMentionTermination("TERMINATE") | MaxMessageTermination(10)

    # The group chat will alternate between the assistant and the code executor.
    group_chat = RoundRobinGroupChat(
        [assistant, code_executor],
        termination_condition=termination
    )

    # `run_stream` returns an async generator to stream the intermediate messages.
    stream = group_chat.run_stream(task="Write a python script to print 'Hello, world!'")

    # `Console` is a simple UI to display the stream
    await Console(stream)

    # Close the connection to the model client
    await model_client.close()

await main()

---------- TextMessage (user) ----------
Write a python script to print 'Hello, world!'
---------- TextMessage (assistant) ----------
```python
print("Hello, world!")
```
---------- TextMessage (code_executor) ----------
Hello, world!

---------- TextMessage (assistant) ----------
TERMINATE


## 10.Tool Use

In `v0.4`, you really just need one agent – the [AssistantAgent](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.agents.html#autogen_agentchat.agents.AssistantAgent) – to handle both the tool calling and tool execution.

When using tool-equipped agents inside a group chat such as [RoundRobinGroupChat](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.teams.html#autogen_agentchat.teams.RoundRobinGroupChat), you simply do the same as above to add tools to the agents, and create a group chat with the agents.

```python
import asyncio
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage

def get_weather(city: str) -> str: # Async tool is possible too.
    return f"The weather in {city} is 72 degree and sunny."

async def main() -> None:
    model_client = OpenAIChatCompletionClient(model="gpt-4o", seed=42, temperature=0)
    assistant = AssistantAgent(
        name="assistant",
        system_message="You are a helpful assistant. You can call tools to help user.",
        model_client=model_client,
        tools=[get_weather],
        reflect_on_tool_use=True, # Set to True to have the model reflect on the tool use, set to False to return the tool call result directly.
    )
    while True:
        user_input = input("User: ")
        if user_input == "exit":
            break
        response = await assistant.on_messages([TextMessage(content=user_input, source="user")], CancellationToken())
        print("Assistant:", response.chat_message.to_text())
    await model_client.close()

asyncio.run(main())
```

## 11.Chat Result

In `v0.4`, you get a [TaskResult](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.base.html#autogen_agentchat.base.TaskResult) object from a `run` or `run_stream` method. The [TaskResult](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.base.html#autogen_agentchat.base.TaskResult) object contains the `messages` which is the message history of the chat, including both agents’ private (tool calls, etc.) and public messages.

## 12.Conversion between v0.2 and v0.4 Messages

You can use the following conversion functions to convert between a v0.4 message in [autogen_agentchat.base.TaskResult.messages](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.base.html#autogen_agentchat.base.TaskResult.messages) and a v0.2 message in `ChatResult.chat_history`.

```python
from typing import Any, Dict, List, Literal

from autogen_agentchat.messages import (
    BaseAgentEvent,
    BaseChatMessage,
    HandoffMessage,
    MultiModalMessage,
    StopMessage,
    TextMessage,
    ToolCallExecutionEvent,
    ToolCallRequestEvent,
    ToolCallSummaryMessage,
)
from autogen_core import FunctionCall, Image
from autogen_core.models import FunctionExecutionResult


def convert_to_v02_message(
    message: BaseAgentEvent | BaseChatMessage,
    role: Literal["assistant", "user", "tool"],
    image_detail: Literal["auto", "high", "low"] = "auto",
) -> Dict[str, Any]:
    """Convert a v0.4 AgentChat message to a v0.2 message.

    Args:
        message (BaseAgentEvent | BaseChatMessage): The message to convert.
        role (Literal["assistant", "user", "tool"]): The role of the message.
        image_detail (Literal["auto", "high", "low"], optional): The detail level of image content in multi-modal message. Defaults to "auto".

    Returns:
        Dict[str, Any]: The converted AutoGen v0.2 message.
    """
    v02_message: Dict[str, Any] = {}
    if isinstance(message, TextMessage | StopMessage | HandoffMessage | ToolCallSummaryMessage):
        v02_message = {"content": message.content, "role": role, "name": message.source}
    elif isinstance(message, MultiModalMessage):
        v02_message = {"content": [], "role": role, "name": message.source}
        for modal in message.content:
            if isinstance(modal, str):
                v02_message["content"].append({"type": "text", "text": modal})
            elif isinstance(modal, Image):
                v02_message["content"].append(modal.to_openai_format(detail=image_detail))
            else:
                raise ValueError(f"Invalid multimodal message content: {modal}")
    elif isinstance(message, ToolCallRequestEvent):
        v02_message = {"tool_calls": [], "role": "assistant", "content": None, "name": message.source}
        for tool_call in message.content:
            v02_message["tool_calls"].append(
                {
                    "id": tool_call.id,
                    "type": "function",
                    "function": {"name": tool_call.name, "args": tool_call.arguments},
                }
            )
    elif isinstance(message, ToolCallExecutionEvent):
        tool_responses: List[Dict[str, str]] = []
        for tool_result in message.content:
            tool_responses.append(
                {
                    "tool_call_id": tool_result.call_id,
                    "role": "tool",
                    "content": tool_result.content,
                }
            )
        content = "\n\n".join([response["content"] for response in tool_responses])
        v02_message = {"tool_responses": tool_responses, "role": "tool", "content": content}
    else:
        raise ValueError(f"Invalid message type: {type(message)}")
    return v02_message


def convert_to_v04_message(message: Dict[str, Any]) -> BaseAgentEvent | BaseChatMessage:
    """Convert a v0.2 message to a v0.4 AgentChat message."""
    if "tool_calls" in message:
        tool_calls: List[FunctionCall] = []
        for tool_call in message["tool_calls"]:
            tool_calls.append(
                FunctionCall(
                    id=tool_call["id"],
                    name=tool_call["function"]["name"],
                    arguments=tool_call["function"]["args"],
                )
            )
        return ToolCallRequestEvent(source=message["name"], content=tool_calls)
    elif "tool_responses" in message:
        tool_results: List[FunctionExecutionResult] = []
        for tool_response in message["tool_responses"]:
            tool_results.append(
                FunctionExecutionResult(
                    call_id=tool_response["tool_call_id"],
                    content=tool_response["content"],
                    is_error=False,
                    name=tool_response["name"],
                )
            )
        return ToolCallExecutionEvent(source="tools", content=tool_results)
    elif isinstance(message["content"], list):
        content: List[str | Image] = []
        for modal in message["content"]:  # type: ignore
            if modal["type"] == "text":  # type: ignore
                content.append(modal["text"])  # type: ignore
            else:
                content.append(Image.from_uri(modal["image_url"]["url"]))  # type: ignore
        return MultiModalMessage(content=content, source=message["name"])
    elif isinstance(message["content"], str):
        return TextMessage(content=message["content"], source=message["name"])
    else:
        raise ValueError(f"Unable to convert message: {message}")
```

## 13.Group Chat

In v0.4, you can use the [RoundRobinGroupChat](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.teams.html#autogen_agentchat.teams.RoundRobinGroupChat)

In [12]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient


async def main() -> None:

    model_client = OpenAIChatCompletionClient(
        model="gpt-4.1-nano",
        seed=42,
        temperature=0,
        api_key=os.getenv("OPENAI_API_KEY")
    )

    writer = AssistantAgent(
        name="writer",
        description="A writer.",
        system_message="You are a writer.",
        model_client=model_client
    )

    critic = AssistantAgent(
        name="critic",
        description="A critic.",
        system_message="You are a critic, provide feedback on the writing. Reply only 'APPROVE' if the task is done.",
        model_client=model_client
    )

    # The termination condition is a text termination,
    # which will cause the chat to terminate when the text "APPROVE" is received.
    termination = TextMentionTermination("APPROVE")

    # The group chat will alternate between the writer and the critic.
    group_chat = RoundRobinGroupChat([writer, critic],
                                     termination_condition=termination,
                                     max_turns=12
                                    )

    # `run_stream` returns an async generator to stream the intermediate messages.
    stream = group_chat.run_stream(task="Write a short story about a robot that discovers it has feelings.")
    
    # `Console` is a simple UI to display the stream.
    await Console(stream)
    
    # Close the connection to the model client.
    await model_client.close()

await main()

---------- TextMessage (user) ----------
Write a short story about a robot that discovers it has feelings.
---------- TextMessage (writer) ----------
In the bustling city of Neo-Terra, amidst towering skyscrapers and humming machinery, there was a robot named Axiom. Designed for maintenance, Axiom spent its days repairing circuits and fixing broken systems, its existence defined by logic and precision.

One evening, as the sun dipped below the horizon, casting a warm amber glow, Axiom was assigned to repair a malfunctioning fountain in the city square. As it worked, a small child approached, clutching a worn-out teddy bear. The child’s eyes sparkled with curiosity and innocence.

Axiom extended its mechanical arm to assist, but something unusual happened. As it watched the child giggle at the water splashing from the fountain, a strange warmth spread through its circuits—a sensation it had never experienced before. It felt a flicker of something akin to happiness, a gentle pulse that r

## 14.Group Chat with Resume

In `v0.4`, you can simply call `run` or `run_stream` again with the same group chat object to resume the chat. To export and load the state, you can use `save_state` and `load_state` methods. See [Group Chat with Resume](https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/migration-guide.html#group-chat-with-resume) for an example.

In [16]:
import json
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination

def create_team(model_client: OpenAIChatCompletionClient)->RoundRobinGroupChat:
    writer = AssistantAgent(
        name="writer",
        description="A writer.",
        system_message="You are a writer.",
        model_client=model_client
    )

    critic = AssistantAgent(
        name="critic",
        description="A critic.",
        system_message="You are a critic, provide feedback on the writing. Reply only 'APPROVE' if the task is done.",
        model_client=model_client,
    )

    # The termination condition is a text termination,
    # which will cause the chat to terminate when the text "APPROVE" is received.
    termination=TextMentionTermination("APPROVE")

    # The group chat will alternate between the writer and the critic.
    group_chat=RoundRobinGroupChat(
        [writer, critic],
        termination_condition=termination
    )

    return group_chat

async def main()->None:
    
    model_client = OpenAIChatCompletionClient(
        model="gpt-4.1-nano",
        seed=42,
        temperature=0,
        api_key=os.getenv("OPENAI_API_KEY")
    )

    # Create team.
    group_chat = create_team(model_client)

    # `run_stream` returns an async generator to stream the intermediate messages.
    stream = group_chat.run_stream(task="Write a short story about a robot that discovers it has feelings.")
    
    # `Console` is a simple UI to display the stream.
    await Console(stream)

    # Save the state of the group chat and all participants.
    state = await group_chat.save_state()
    with open("group_chat_state.json","w") as f:
        json.dump(state,f,default=str)

    # Create a new team with the same participants configuration.
    group_chat = create_team(model_client)

    # Load the state of the group chat and all participants.
    with open("group_chat_state.json", "r") as f:
        state = json.load(f)
    await group_chat.load_state(state)

    # Resume the chat.
    stream = group_chat.run_stream(task="Translate the story into Chinese.")
    await Console(stream)

    # Close the connection to the model client.
    await model_client.close()

await main()

---------- TextMessage (user) ----------
Write a short story about a robot that discovers it has feelings.
---------- TextMessage (writer) ----------
In the bustling city of Neo-Terra, amidst towering skyscrapers and humming machinery, there was a robot named Axiom. Designed for maintenance, Axiom spent its days repairing circuits and cleaning streets, its movements precise and efficient. But unlike other robots, Axiom had a curious spark—an anomaly in its programming that made it pause and wonder.

One evening, as the sun dipped below the horizon, casting a golden glow over the city, Axiom found itself near a park. Children’s laughter echoed through the air, and the scent of blooming flowers wafted on the breeze. As Axiom watched, a girl dropped her balloon, and it floated away, caught in the wind. Without thinking, Axiom reached out and gently caught it, returning it to her with a soft whirr.

The girl smiled brightly. “Thank you,” she said, her eyes shining.

Axiom felt a strange wa

## 15.Group Chat with Custom Selector (Stateflow)

In [19]:
from typing import Sequence
from autogen_agentchat.ui import Console
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.messages import BaseAgentEvent, BaseChatMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Note: This example uses mock tools instead of real APIs for demonstration purposes
def search_web_tool(query: str) -> str:
    if "2006-2007" in query:
        return """Here are the total points scored by Miami Heat players in the 2006-2007 season:
        Udonis Haslem: 844 points
        Dwayne Wade: 1397 points
        James Posey: 550 points
        ...
        """
    elif "2007-2008" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2007-2008 is 214."
    elif "2008-2009" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2008-2009 is 398."
    return "No data found."


def percentage_change_tool(start: float, end: float) -> float:
    return ((end - start) / start) * 100

def create_team(model_client : OpenAIChatCompletionClient) -> SelectorGroupChat:
    
    planning_agent = AssistantAgent(
        "PlanningAgent",
        description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
        model_client=model_client,
        system_message="""
        You are a planning agent.
        Your job is to break down complex tasks into smaller, manageable subtasks.
        Your team members are:
            Web search agent: Searches for information
            Data analyst: Performs calculations

        You only plan and delegate tasks - you do not execute them yourself.

        When assigning tasks, use this format:
        1. <agent> : <task>

        After all tasks are complete, summarize the findings and end with "TERMINATE".
        """,
    )

    web_search_agent = AssistantAgent(
        "WebSearchAgent",
        description="A web search agent.",
        tools=[search_web_tool],
        model_client=model_client,
        system_message="""
        You are a web search agent.
        Your only tool is search_tool - use it to find information.
        You make only one search call at a time.
        Once you have the results, you never do calculations based on them.
        """,
    )

    data_analyst_agent = AssistantAgent(
        "DataAnalystAgent",
        description="A data analyst agent. Useful for performing calculations.",
        model_client=model_client,
        tools=[percentage_change_tool],
        system_message="""
        You are a data analyst.
        Given the tasks you have been assigned, you should analyze the data and provide results using the tools provided.
        """,
    )

    # The termination condition is a combination of text mention termination and max message termination.
    text_mention_termination = TextMentionTermination("TERMINATE")
    max_messages_termination = MaxMessageTermination(max_messages=25)
    termination = text_mention_termination | max_messages_termination

    # The selector function is a function that takes the current message thread of the group chat
    # and returns the next speaker's name. If None is returned, the LLM-based selection method will be used.
    def selector_func(messages: Sequence[BaseAgentEvent | BaseChatMessage]) -> str | None:
        if messages[-1].source != planning_agent.name:
            return planning_agent.name # Always return to the planning agent after the other agents have spoken.
        return None

    team = SelectorGroupChat(
        [planning_agent, web_search_agent, data_analyst_agent],
        model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"), # Use a smaller model for the selector.
        termination_condition=termination,
        selector_func=selector_func,
    )
    return team


async def main() -> None:

    model_client = OpenAIChatCompletionClient(
        model="gpt-4.1-nano",
        seed=42,
        temperature=0,
        api_key=os.getenv("OPENAI_API_KEY")
    )
    
    team = create_team(model_client)
    
    task = "Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?"
    
    await Console(team.run_stream(task=task))

await main()

---------- TextMessage (user) ----------
Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- TextMessage (PlanningAgent) ----------
1. Web search agent: Search for the Miami Heat player with the highest points in the 2006-2007 season.
2. Web search agent: Find the total rebounds for that player in the 2007-2008 and 2008-2009 seasons.
3. Data analyst: Calculate the percentage change in total rebounds between the 2007-2008 and 2008-2009 seasons for that player.
---------- ToolCallRequestEvent (WebSearchAgent) ----------
[FunctionCall(id='call_cb2jP9e07VGDewysicWgsvjm', arguments='{"query": "Miami Heat player with the highest points in the 2006-2007 season"}', name='search_web_tool'), FunctionCall(id='call_Cw3Ld4aynWpbsP7lLt6vgxJA', arguments='{"query": "Total rebounds of Miami Heat player in the 2007-2008 season"}', name='search_web_tool'), FunctionC

## 16.Nested Chat

In `v0.4`, nested chat is an implementation detail of a custom agent. You can create a custom agent that takes a team or another agent as a parameter and implements the on_messages method to trigger the nested team or agent. It is up to the application to decide how to pass or transform the messages from and to the nested team or agent.

The following example shows a simple nested chat that counts numbers.

```python
import asyncio
from typing import Sequence
from autogen_core import CancellationToken
from autogen_agentchat.agents import BaseChatAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.messages import TextMessage, BaseChatMessage
from autogen_agentchat.base import Response

class CountingAgent(BaseChatAgent):
    """An agent that returns a new number by adding 1 to the last number in the input messages."""
    async def on_messages(self, messages: Sequence[BaseChatMessage], cancellation_token: CancellationToken) -> Response:
        if len(messages) == 0:
            last_number = 0 # Start from 0 if no messages are given.
        else:
            assert isinstance(messages[-1], TextMessage)
            last_number = int(messages[-1].content) # Otherwise, start from the last number.
        return Response(chat_message=TextMessage(content=str(last_number + 1), source=self.name))

    async def on_reset(self, cancellation_token: CancellationToken) -> None:
        pass

    @property
    def produced_message_types(self) -> Sequence[type[BaseChatMessage]]:
        return (TextMessage,)

class NestedCountingAgent(BaseChatAgent):
    """An agent that increments the last number in the input messages
    multiple times using a nested counting team."""
    def __init__(self, name: str, counting_team: RoundRobinGroupChat) -> None:
        super().__init__(name, description="An agent that counts numbers.")
        self._counting_team = counting_team

    async def on_messages(self, messages: Sequence[BaseChatMessage], cancellation_token: CancellationToken) -> Response:
        # Run the inner team with the given messages and returns the last message produced by the team.
        result = await self._counting_team.run(task=messages, cancellation_token=cancellation_token)
        # To stream the inner messages, implement `on_messages_stream` and use that to implement `on_messages`.
        assert isinstance(result.messages[-1], TextMessage)
        return Response(chat_message=result.messages[-1], inner_messages=result.messages[len(messages):-1])

    async def on_reset(self, cancellation_token: CancellationToken) -> None:
        # Reset the inner team.
        await self._counting_team.reset()

    @property
    def produced_message_types(self) -> Sequence[type[BaseChatMessage]]:
        return (TextMessage,)

async def main() -> None:
    # Create a team of two counting agents as the inner team.
    counting_agent_1 = CountingAgent("counting_agent_1", description="An agent that counts numbers.")
    counting_agent_2 = CountingAgent("counting_agent_2", description="An agent that counts numbers.")
    counting_team = RoundRobinGroupChat([counting_agent_1, counting_agent_2], max_turns=5)
    # Create a nested counting agent that takes the inner team as a parameter.
    nested_counting_agent = NestedCountingAgent("nested_counting_agent", counting_team)
    # Run the nested counting agent with a message starting from 1.
    response = await nested_counting_agent.on_messages([TextMessage(content="1", source="user")], CancellationToken())
    assert response.inner_messages is not None
    for message in response.inner_messages:
        print(message)
    print(response.chat_message)

asyncio.run(main())
```

You can take a look at [SocietyOfMindAgent](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.agents.html#autogen_agentchat.agents.SocietyOfMindAgent) for a more complex implementation.

## 17.Sequential Chat

In `v0.2`, sequential chat is supported by using the `initiate_chats` function. It takes input a list of dictionary configurations for each step of the sequence. See [Sequential Chat in v0.2](https://microsoft.github.io/autogen/0.2/docs/tutorial/conversation-patterns#sequential-chats) for more details.

Base on the feedback from the community, the `initiate_chats` function is too opinionated and not flexible enough to support the diverse set of scenarios that users want to implement. We often find users struggling to get the `initiate_chats` function to work when they can easily glue the steps together usign basic Python code. Therefore, in `v0.4`, we do not provide a built-in function for sequential chat in the AgentChat API.

Instead, you can create an event-driven sequential workflow using the Core API, and use the other components provided the AgentChat API to implement each step of the workflow. See an example of sequential workflow in the [Core API Tutorial](https://microsoft.github.io/autogen/stable/user-guide/core-user-guide/design-patterns/sequential-workflow.html).

We recognize that the concept of workflow is at the heart of many applications, and we will provide more built-in support for workflows in the future.

## 18.GPTAssistantAgent

In `v0.2`, `GPTAssistantAgent` is a special agent class that is backed by the OpenAI Assistant API.

In `v0.4`, the equivalent is the [OpenAIAssistantAgent](https://microsoft.github.io/autogen/stable/reference/python/autogen_ext.agents.openai.html#autogen_ext.agents.openai.OpenAIAssistantAgent) class. It supports the same set of features as the `GPTAssistantAgent` in `v0.2` with more such as customizable threads and file uploads. See [OpenAIAssistantAgent](https://microsoft.github.io/autogen/stable/reference/python/autogen_ext.agents.openai.html#autogen_ext.agents.openai.OpenAIAssistantAgent) for more details.

## 19.Long Context Handling

In `v0.4`, we introduce the [ChatCompletionContext](https://microsoft.github.io/autogen/stable/reference/python/autogen_core.model_context.html#autogen_core.model_context.ChatCompletionContext) base class that manages message history and provides a virtual view of the history. Applications can use built-in implementations such as [BufferedChatCompletionContext](https://microsoft.github.io/autogen/stable/reference/python/autogen_core.model_context.html#autogen_core.model_context.BufferedChatCompletionContext) to limit the message history sent to the model, or provide their own implementations that creates different virtual views.

To use [BufferedChatCompletionContext](https://microsoft.github.io/autogen/stable/reference/python/autogen_core.model_context.html#autogen_core.model_context.BufferedChatCompletionContext) in an [AssistantAgent](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.agents.html#autogen_agentchat.agents.AssistantAgent) in a chatbot scenario.

In this example, the chatbot can only read the last 10 messages in the history.

```python
import asyncio
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.agents import AssistantAgent
from autogen_core import CancellationToken
from autogen_core.model_context import BufferedChatCompletionContext
from autogen_ext.models.openai import OpenAIChatCompletionClient

async def main() -> None:
    model_client = OpenAIChatCompletionClient(model="gpt-4o", seed=42, temperature=0)

    assistant = AssistantAgent(
        name="assistant",
        system_message="You are a helpful assistant.",
        model_client=model_client,
        model_context=BufferedChatCompletionContext(buffer_size=10), # Model can only view the last 10 messages.
    )
    while True:
        user_input = input("User: ")
        if user_input == "exit":
            break
        response = await assistant.on_messages([TextMessage(content=user_input, source="user")], CancellationToken())
        print("Assistant:", response.chat_message.to_text())
    
    await model_client.close()

asyncio.run(main())
```


## 20.Observability and Control

In `v0.4` AgentChat, you can observe the agents by using the `on_messages_stream` method which returns an async generator to stream the inner thoughts and actions of the agent. For teams, you can use the `run_stream` method to stream the inner conversation among the agents in the team. Your application can use these streams to observe the agents and teams in real-time.

Both the `on_messages_stream` and `run_stream` methods takes a [CancellationToken](https://microsoft.github.io/autogen/stable/reference/python/autogen_core.html#autogen_core.CancellationToken) as a parameter which can be used to cancel the output stream asynchronously and stop the agent or team. For teams, you can also use termination conditions to stop the team when a certain condition is met. See [Termination Condition Tutorial](https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/tutorial/termination.html) for more details.

Unlike the v0.2 which comes with a special logging module, the v0.4 API simply uses Python’s logging module to log events such as model client calls. See [Logging](https://microsoft.github.io/autogen/stable/user-guide/core-user-guide/framework/logging.html) in the Core API documentation for more details.

## 21.Code Executors

The code executors in `v0.2` and `v0.4` are nearly identical except the `v0.4` executors support async API. You can also use [CancellationToken](https://microsoft.github.io/autogen/stable/reference/python/autogen_core.html#autogen_core.CancellationToken) to cancel a code execution if it takes too long. See [Command Line Code Executors Tutorial](https://microsoft.github.io/autogen/stable/user-guide/core-user-guide/components/command-line-code-executors.html) in the Core API documentation.

We also added `ACADynamicSessionsCodeExecutor` that can use Azure Container Apps (ACA) dynamic sessions for code execution. See [ACA Dynamic Sessions Code Executor Docs](https://microsoft.github.io/autogen/stable/user-guide/extensions-user-guide/azure-container-code-executor.html).